In [1]:
import random
import time

class Stone: # 棋子
    def __init__(self,x,y,c):
        self.x = x
        self.y = y
        self.c = c # 标识黑白子
        self.pos = (x,y)
    def __call__(self,x,y,c):
        self.x = x
        self.y = y
        self.c = c 
        self.pos = (x,y)
        return self       
    def __str__(self):
        return str(self.c) + "_" + str(self.x) + "_" + str(self.y)
        
          

class Group: # 棋子整体串
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.stones = set() # 棋子集合
        self.airs = set() # 气点集合
        self.liberty = 0 # 气数
        self.opponents = set() #棋子整体周围的对方棋子整体
        
    def __str__(self):
        res = "{"
        
        for s in self.stones:
            res += str(s)
            res += " "
            
        res += "} Lives:"
        res += str(self.liberty)
        
        res += " Airs:" + str(self.airs)
        return res
        
    def add_stone(self,stone):
        self.stones.add(stone)
        
    def del_stone(self,stone):
        if stone in self.stones:
            self.reset()
            
    def add_op(self,op):
        self.opponents.add(op)
        
    def del_op(self,op):
        if op in self.opponents:
            self.opponents.remove(op)
        
    def add_air(self,air):
        self.airs.add(air)
        self.liberty = len(self.airs)
        
    def del_air(self,air):
        if air in self.airs:
            self.airs.remove(air)
            self.liberty = len(self.airs)
            
    def merge(self,group):
        self.stones = self.stones.union(group.stones)
        self.airs = self.airs.union(group.airs)
        self.liberty = len(self.airs)
        
        return self

class Go:
    def __init__(self,size = 19):
        self.size = size
        self.ws = set()
        self.bs = set()
        self.wg = set()
        self.bg = set()
        self.hands = 0
        self.player = 'B'
        self.records = dict()
        self.ko = None # 劫
        
        self.w_no = set() # 白棋禁入点
        self.b_no = set() # 黑棋禁入点
        self.value = .52382 # 黑棋获胜的初始概率
        
        self.take_info = dict() # 发生提子的相关信息
        self.take_history = list() # 提子的历史记录信息
        
    def render(self): # OK
        print(">>Hand:%3d "%self.hands +"--."*((self.size+1)-4))
#        print("X:BLACK\nO:WHITE\n.:SPACE")
        for k in range(self.size):
            print("%-2d"%(k+1),end = ' ')
        print("")
        
        for i in range(self.size):
            for j in range(self.size):
                c = '.'
                if (i,j) in self.ws:
                    c = 'O'
                elif (i,j) in self.bs:
                    c = 'X'
                print(c,end='  ')
        
            print(" "+str(i+1))
            
        print("BLACK Win: %.2f%%"%(self.value*100))
#         print("v-."*(self.size+1))
#         print()
        
#        self.display()
    
    def _is_legal(self,pos): # OK
        x = pos[0]
        y = pos[1]
        if x<0 or y<0 or x>=self.size or y>=self.size:
            return False
            
        if pos in self.ws:
            return False
        if pos in self.bs:
            return False
        if pos == self.ko:
            return False
            
        ####
        ## 判断是否在禁入点上
        if self.player == 'B':
            if pos in self.b_no:
                return False
            else:
                return True
                
        else:
            if pos in self.w_no:
                return False
            else:
                return True
        ####   
        return True     
        
    def _is_take(self,pos): # OK 判断是否需要提子
        # 需要逻辑判断
        # 例如，当前棋手黑'B',则判断self.wg 中是否有某个串g的airs属性中只含有1个点且该点为pos
        # 如果是，则需要提子，返回True，如果不是，返回False
        
        if self.player == 'B':
            for g in self.wg:
                if g.liberty == 1:
                    if pos in g.airs:
                        return True
        else:
            for g in self.bg:
                if g.liberty == 1:
                    if pos in g.airs:
                        return True
                        
        return False
        
    def _take(self,pos): #  提子
        # 需要更改ws 或 bs中的信息  
        # 需要记录提走了哪些子
        self.take_info['hands'] = self.hands
        self.take_info['player'] = self.player
        self.take_info['pos'] = pos
        self.take_info['stones'] = list()
        if self.player == 'B':
            for g in self.wg:
                if g.liberty == 1:
                    if pos in g.airs:
                        for stone in g.stones:
                            self.ws.remove(stone.pos)
                            self.take_info['stones'].append(stone.pos)
                            
                            
        else:
            for g in self.bg:
                if g.liberty == 1:
                    if pos in g.airs:
                        for stone in g.stones:
                            self.bs.remove(stone.pos)
                            self.take_info['stones'].append(stone.pos)
                            
        self.take_history.append(self.take_info) # 把本次提子信息加入提子历史记录中
        
    def display(self):
        print("当前的白串:")
        for g in self.wg:
            print(g,'\n')
            
        print("当前的黑串:")
        for g in self.bg:
            print(g,'\n')
                 
        print("当前的白棋")
        print(self.ws,'\n')
        
        print("当前的黑棋")
        print(self.bs,'\n')
        
        print("当前的死子")
        print(self.take_history,'\n')
        
        print("当前的劫")
        print(self.ko,'\n')
        
        print("当前的白禁入点")
        print(self.w_no,'\n')
        
        print("当前的黑禁入点")
        print(self.b_no,'\n')
        
        print("当前的棋谱")
        print(self.records)
        
        print("当前的棋手:",self.player)
        
        print("当前的黑子数：",len(self.bs),"白子数",len(self.ws),"黑禁入点数：",len(self.b_no),\
              "白禁入点数：",len(self.w_no),"总数：",len(self.bs)+len(self.ws)+len(self.b_no)+len(self.w_no))

    
    """
    update方法是围棋的核心方法，根据当前局面更新相关信息(wg,bg,ko,w_no,b_no)
    
    递归扫描每个点，并对扫描过的点做标记，直到递归结束，生成一个group
    如果该点已经做过标记，则跳过，如果没有标记，则再递归生成一个group。这样就完成了 wg 和 bg的更新
    
    
    对于劫ko的计算
    如果take_info 中有内容，则发生了提子，根据take_info计算ko点
    提了对方一子，如果对方在死子的位置落子,也能提走本方的落子，且本方落子的所在的组的长度为1，则该点为ko点
    
    
    对于禁入点的计算
    比如 计算 黑禁入点 b_no
    扫描每个非落子点，如果该点的四周都有子，则考虑四周各串的liberty == 1的情况
    如果该group为黑串，且四周的白串的liberty都>1,则该点为黑禁入点
    
    同样的思路 考虑该非落子点是否为白禁入点 w_no
    
    """
    def update(self):
        self.wg = set()
        self.bg = set()
        self.ko = None
        self.b_no = set()
        self.w_no = set()
        
        # 生成标记
        labels = dict()
        for i in range(self.size):
            for j in range(self.size):
                    labels[(i,j)] = 0  # 初始化为未扫描标记0
                    
        """
        递归扫描每个点，并对扫描过的点做标记，直到递归结束，生成一个group
        如果该点已经做过标记，则跳过，如果没有标记，则再递归生成一个group。这样就完成了 wg 和 bg的更新
        """                   
        for i in range(self.size):
            for j in range(self.size):
                point = (i,j) #扫描这个点
                if labels[point] == 0: # 没有扫描
                    labels[point] = 1 # 标识为已扫描
                    stone = None
                    color = None
                    if point in self.ws:
                        stone = Stone(point[0], point[1],'W')
                        color = 'W'
                    elif point in self.bs:
                        stone = Stone(point[0], point[1],'B')
                        color = 'B'
                    if stone:
                        group = Group()
                        group.add_stone(stone)
                        
                        w_group_set,b_group_set,airs_set = self.get_sets(point)
                        
                        for air in airs_set: # 加入气点
                            group.add_air(air)
                        
                        if color == 'B':
                            for g in b_group_set:
                                
                                group.merge(g)
#                                print("TBT",group)
                                self.bg.remove(g)
                                
                                for s in g.stones:
                                    labels[s.pos] = 1
                                self.bg.add(group)

                            self.bg.add(group)

                                
                        elif color == 'W':
                            for g in w_group_set:
#                                print("TWT",g)
                                group.merge(g)
                                self.wg.remove(g)
                            
                                for s in g.stones:
                                    labels[s.pos] = 1

                                self.wg.add(group)
        
                            self.wg.add(group)
                         
                        
        """                    
        对于劫ko的计算
        如果take_info 中有内容，则发生了提子，根据take_info计算ko点
        提了对方一子，如果对方在死子的位置落子,也能提走本方的落子，且本方落子的所在的组的长度为1，则该点为ko点
        """
        if self.take_info:
            if len(self.take_info['stones']) == 1:
#                print("提了一个")
                for stone in self.take_info['stones']:
                    take_point = stone
                pos = self.take_info['pos']
                group,_ = self.query_group(pos)
#                print(group)
                if len(group.stones) == 1:
                    if group.liberty == 1:
                        self.ko = take_point
                        
                        
        """
        对于禁入点的计算
        比如 计算 黑禁入点 b_no
        扫描每个非落子点，如果该点的四周都有子，则考虑四周各串的liberty == 1的情况
        如果该group为黑串，且四周的白串的liberty都>1,且四周的黑串的liberty都=1,则该点为黑禁入点
        """
        for i in range(self.size):
            for j in range(self.size):
                point = (i,j)
                if point not in self.ws and point not in self.bs:
                    cur_player = self.player
                    if self.player == 'W':
                        self.player = 'B'
                        
                    take_flag = self._is_take(point)
                    if take_flag:
                        # 如何可以提子，则此点肯定不是黑禁入点
                        pass
                    else:
                        # 如果不能提子，且该点周围的黑串的气都为1，则该点为黑禁入点
                        w_group_set,b_group_set,airs_set = self.get_sets(point)
                        b_flag = True
                        for bg in b_group_set:
                            if bg.liberty > 1:
                                b_flag = False
                                
                        if b_flag and not airs_set:
                            self.b_no.add(point)
                           
                ###############
                           
                    if self.player == 'B':
                        self.player = 'W'
                    take_flag = self._is_take(point)
                    if take_flag:
                        # 如何可以提子，则此点肯定不是白禁入点
                        pass
                    else:
                        # 如果不能提子，且该点周围的白串的气都为1，则该点为白禁入点
                        w_group_set,b_group_set,airs_set = self.get_sets(point)
                        w_flag = True
                        for wg in w_group_set:
                            if wg.liberty >1:
                                w_flag = False
    
                        if w_flag and  not airs_set:
                           self.w_no.add(point)   
                           
                         
                    self.player = cur_player   
           
     
    """
    围棋的核心方法，根据当前局面更新相关信息(wg,bg,ko,w_no,b_no)
    """
    
     
    def get_neighbors(self,pos): # 得到pos周围的坐标
        x = pos[0]
        y = pos[1]
        if x<0 or y<0 or x>=self.size or y>=self.size:
            return list()
        xs = [x-1,x+1]
        ys = [y-1,y+1]
        
        if x-1 <0:
            xs.remove(x-1)
        if x+1>self.size-1:
            xs.remove(x+1)
            
        if y-1 <0:
            ys.remove(y-1)
        if y+1>self.size-1:
            ys.remove(y+1)
            
        results = []

        for r in xs:
            results.append((r,y))
        for c in ys:
            results.append((x,c))
             
        return results
        
    def query_pos(self,pos): # 查询某个位置属于黑棋 白棋，空地
        if pos in self.ws:
            return 1 # 白子
        elif pos in self.bs:
            return 0 # 黑子
        else:
            return -1 #空地
            
    def query_group(self, pos): #  查询某个位置属于哪个group
        for g in self.bg:
            for s in g.stones:
                if pos[0] == s.x and pos[1] == s.y:
                    return g,'B'
        for g in self.wg:
            for s in g.stones:
                if pos[0] == s.x and pos[1] == s.y:
                    return g,'W'
                    
        return None,'E'
    
        
    def get_sets(self,pos): # 得到该pos四周的组
        neighbors = self.get_neighbors(pos)
        w_group_set = set()
        b_group_set = set()
        airs_set = set()
        if neighbors:
            for neighbor in neighbors:
                if self.query_pos(neighbor) == -1:
                    airs_set.add(neighbor)
                else:
                    g,c = self.query_group(neighbor)
                    if c == 'B':
                        b_group_set.add(g)
                    elif c == 'W':
                        w_group_set.add(g)
                    
        return w_group_set,b_group_set,airs_set
    
    """
    落子方法
    """    
    def move(self,pos):
        #0 手数加1
        self.hands += 1 
        #1 判断是否合法
        if not self._is_legal(pos):
            cur_pos = 'PASS'
#            print('PASS',pos)
        else:
            cur_pos = str(pos)

            #2 判断是否提子
            if self._is_take(pos):
                self._take(pos)
#                print("TAKE",pos)
                # 是，提走棋子
            else:
                self.take_info = dict()
            
            
            # 3.1 添加当前棋子到黑子集合或白子集合
            if self.player == 'B':
                self.bs.add(pos)
            else:
                self.ws.add(pos)
                
            # 3.2 根据当前局面 更新相关信息, ***** 核心 *****
        self.update()
            
        #4 更新记录棋谱
        self.records[self.hands] = cur_pos
        #5 切换棋手
        self.player = 'W' if self.player == 'B' else 'B'

    def who_win(self):
        return 1 # 表示黑棋获胜
        return 0 # 表示无胜负
        return -1 # 表示白棋获胜
        return -2  # 表示暂时不能断定胜负
    
    
go = Go()
import pygame
pygame.init()
clock = pygame.time.Clock()

env_size = go.size -1
block_width, block_height = 32,32
margin = 64
width, height = block_width *env_size ,block_height*env_size
bg_size = width + margin, height + margin


red =(255,0,0)
yellow =(255,206,148)
dark_yellow = (212,208,200)
black =(0,0,0)
white = (255,255,255)
gray = (128,128,128)
green =(0,255,0)
blue =(0,0,255)

screen = pygame.display.set_mode(bg_size)
borad = pygame.Surface((width,height))
pygame.display.set_caption("Go   @Cnleeds --R2018")

# 画线
def draw_lines():
    pygame.draw.line(borad,black,(0,0),(width,0),2)
    pygame.draw.line(borad,black,(0,height-2),(width,height-2),2)
    
    pygame.draw.line(borad,black,(0,0),(0,height),2)
    pygame.draw.line(borad,black,(width-2,0),(width-2,height),2)
    
    pygame.draw.circle(borad,black,(width//2,height//2),3,0)
    
    pygame.draw.circle(borad,black,(width//2,height//6),3,0)
    pygame.draw.circle(borad,black,(width//2,height - height//6),3,0)
    
    pygame.draw.circle(borad,black,(width//6,height//2),3,0)
    pygame.draw.circle(borad,black,(width - width//6,height//2),3,0)
    
    pygame.draw.circle(borad,black,(width//6,height//6),3,0)
    pygame.draw.circle(borad,black,(width - width//6,height - height//6),3,0)
    
    pygame.draw.circle(borad,black,(width//6,height - height//6),3,0)
    pygame.draw.circle(borad,black,(width - width//6,height//6),3,0)
    
    for i in range(env_size-1):
        pygame.draw.line(borad, black, (0,(i+1)*block_height),(width,(i+1)*block_height),1)
        pygame.draw.line(borad, black, ((i+1)*block_width,0),((i+1)*block_width,height),1)
    

def get_xy(pos):
    px = pos[0]
    py = pos[1]
    
    x = (px - margin/2) // block_height
    y = (py - margin/2) // block_width

    mx = (px - margin) % block_height
    my = (py - margin) % block_width 
    if mx > block_height/2:
        x += 1
    if my > block_width/2:
        y += 1
     
    x = int(x)
    y = int(y)
    return (y,x)
    
    
def draw_circle(pos,style):
    if style == 'B':
        pygame.draw.circle(screen,black,(margin//2 + pos[1]*block_width, margin//2 + pos[0]*block_width),block_height//2 - 1,0)
    else:
        pygame.draw.circle(screen,white,(margin//2 + pos[1]*block_width, margin//2 + pos[0]*block_width),block_height//2 - 1,0)
    


def main():
    gameover = False
    
    borad.fill(yellow)
    draw_lines()
    
    screen.fill(dark_yellow)
    screen.blit(borad,(margin/2,margin/2))
    
    while not gameover:
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                gameover = True
            
            if event.type == pygame.MOUSEBUTTONUP:
                screen.fill(dark_yellow)
                screen.blit(borad,(margin/2,margin/2))
                
                borad.fill(yellow)
                draw_lines()
                
                pos = pygame.mouse.get_pos()
                point = get_xy(pos)

                go.move(point)
                go.render()
                print("%s -> Pos:%s\n"%(go.player,point))   
             
                for point in go.bs:
                    draw_circle(point,'B')
                for point in go.ws:
                    draw_circle(point,'W')
 
        pygame.display.update()
        
        clock.tick(60)
        
if __name__ == "__main__":
    main()
    pygame.quit()
    quit()

>>Hand:  1 --.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.
1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   1
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   2
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   3
.  .  .  .  .  .  .  .  .  X  .  .  .  .  .  .  .  .  .   4
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   5
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   6
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   7
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   8
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   9
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   10
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   11
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   12
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   13
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .   14
.  .  .  .  .  .  .  .  .  .  .  .  .

In [1]:
a =1
a

1

In [2]:
ord('A')

65

In [5]:
print(chr(65))

A


In [9]:
for k in range(19):
    print(" ",chr(k+65),end = ' ')
print()    
print("a=1")

  A   B   C   D   E   F   G   H   I   J   K   L   M   N   O   P   Q   R   S 
a=1


In [14]:
import numpy as np
import re

state = np.zeros((9,9),np.int8)
print(re.sub('[\[\]]', '', np.array_str(state)))
print(state)

0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [15]:
no = {(i,j) for i in range(5) for j in range(5)}

In [17]:
type(no)

set